<img style="float: left;padding: 1.3em" src="https://indico.in2p3.fr/event/18313/logo-786578160.png">  

#  Gravitational Wave Open Data Workshop #2

## Paris, April 8-10 2019 [Université Paris Diderot, France](https://www.univ-paris-diderot.fr/)


### Tutorial 2.4: Running parameter estimation on GW150914 using open data.

This example estimates the non-spinning parameters of the binary black hole system using
commonly used prior distributions.  This will take about 30 minutes to run.

[Click this link to view this tutorial in Google Colaboratory](https://colab.research.google.com/github/gw-odw/odw-2019/blob/master/day_2/Tuto 2.4 Parameter estimation for compact object mergers.ipynb)

More at https://lscsoft.docs.ligo.org/bilby/examples.html

## Installation

In [ ]:
! wget --output-document=requirements.txt 'https://raw.githubusercontent.com/gw-odw/odw-2019/master/requirements.txt'
! pip install -r ./requirements.txt

In [ ]:
from __future__ import division, print_function
import bilby

In [ ]:
outdir = 'outdir'
label = 'GW150914'
time_of_event = bilby.gw.utils.get_event_time(label)

This sets up logging output to understand what bilby is doing

In [ ]:
bilby.core.utils.setup_logger(outdir=outdir, label=label)

Here we import the detector data. This step downloads data from the
LIGO/Virgo open data archives. The data is saved to an `Interferometer`
object (here called `H1` and `L1`). A Power Spectral Density (PSD) estimate
is also generated and saved to the same object. To check the data and PSD
makes sense, for each detector a plot is created in the `outdir` called
H1_frequency_domain_data.png and LI_frequency_domain_data.png. The two
objects are then placed into a list.
For GW170817, 170608 and 170814 add the following line to select clean data:
```python
kwargs = {"tag": 'CLN'}
```

In [ ]:
interferometers = bilby.gw.detector.get_event_data(label)

We now define the prior. We have defined our prior distribution in a file packaged with BILBY. The prior is printed to the terminal at run-time.

In [ ]:
prior = bilby.gw.prior.BBHPriorDict(filename='GW150914.prior')

You can overwrite this using the syntax below in the file, or choose a fixed value by just providing a float value as the prior. To speed-up the analysis, let's run with a simpler, non-spinning model:

In [ ]:
prior['a_1']=0.0
prior['a_2']=0.0
prior['tilt_1']=0.0
prior['tilt_2']=0.0
prior['phi_12']=0.0
prior['phi_jl']=0.0

In this step we define a `waveform_generator`. This is out object which
creates the frequency-domain strain. In this instance, we are using the
`lal_binary_black_hole model` source model. We also pass other parameters:
the waveform approximant and reference frequency.

In [ ]:
waveform_generator = bilby.gw.WaveformGenerator(
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    waveform_arguments={'waveform_approximant': 'IMRPhenomPv2',
                        'reference_frequency': 50})

In this step, we define the likelihood. Here we use the standard likelihood
function, passing it the data and the waveform generator.

In [ ]:
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers, waveform_generator, phase_marginalization=True, priors=prior)


Finally, we run the sampler. This function takes the likelihood and prior
along with some options for how to do the sampling and how to save the data

In [ ]:
result = bilby.run_sampler(likelihood, prior, sampler='dynesty',
                           outdir=outdir, label=label)

In [ ]:
result.plot_corner()